In [3]:
import open3d as o3d
import numpy as np
import torch

In [4]:
print("Testing IO for textured meshes ...")
textured_mesh = o3d.io.read_triangle_mesh("/Users/kristofferkjaer/Desktop/mesh/SPRING0014.obj")
print(textured_mesh)
o3d.io.write_triangle_mesh("copy_of_crate.obj",
                            textured_mesh,
                            write_triangle_uvs=True)
copy_textured_mesh = o3d.io.read_triangle_mesh('copy_of_crate.obj')

Testing IO for textured meshes ...
TriangleMesh with 12500 points and 25000 triangles.


### Import object as 3D mesh

In [5]:
mesh = o3d.io.read_triangle_mesh("/Users/kristofferkjaer/Desktop/mesh/SPRING0074.obj")

### Converts mesh to point cloud and visualise

In [6]:
print("Convert mesh to a point cloud and estimate dimensions")
armadillo = o3d.data.ArmadilloMesh()
mesh.compute_vertex_normals()

pcd = mesh.sample_points_poisson_disk(2048)
diameter = np.linalg.norm(
    np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
#o3d.visualization.draw_geometries([pcd])

Convert mesh to a point cloud and estimate dimensions


In [7]:
dataset = o3d.data.RedwoodIndoorOffice1()

In [ ]:
print("Lol hej")